In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import optimizers
from keras.layers import LSTM,Bidirectional
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support
from keras import backend as K
import numpy as np
import re
from sklearn import preprocessing
from gensim.models import Word2Vec
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import keras
from keras.utils import to_categorical

Using TensorFlow backend.
c:\users\terry\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\terry\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\terry\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16

In [9]:
df=pd.read_csv("bbc-text.csv")
df.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [3]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(df['text'].values)
X=tokenizer.texts_to_sequences(df['text'].values)
word_index=tokenizer.word_index
print(f'Found {len(word_index)}of unique tokens')
X= pad_sequences(X, padding = 'post')

Found 29726of unique tokens


In [4]:
l=df['category'].tolist()
le=preprocessing.LabelEncoder()
y=le.fit_transform(l)
y=to_categorical(y)

In [5]:
print(f'Shape of input:{X.shape}')
print(f'Shape of label:{y.shape}')

Shape of input:(2225, 4491)
Shape of label:(2225, 5)


In [6]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=0)

In [7]:
 #importing pretrained word2vec model
 import gensim
 wvmodel=gensim.models.KeyedVectors.load_word2vec_format('https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz',binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [8]:
embedding_matrix=np.zeros((len(word_index)+1,300))

for word,i in word_index.items():
    try:
        embedding_vector=wvmodel[word]
        embedding_matrix[i]=embedding_vector
        
    except KeyError:
        pass
print(f'The shape of embedding matrix is{embedding_matrix.shape}')

The shape of embedding matrix is(29727, 300)


In [9]:
maxlen=len(X[0])
model=Sequential()
model.add(Embedding(input_dim=len(word_index)+1,output_dim=300,weights=[embedding_matrix],input_length=maxlen))
model.add(Dropout(0.1))
model.add(Bidirectional(LSTM(units=100)))
model.add(Dense(5, activation="softmax"))

In [10]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 4491, 300)         8918100   
_________________________________________________________________
dropout_1 (Dropout)          (None, 4491, 300)         0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               320800    
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 1005      
Total params: 9,239,905
Trainable params: 9,239,905
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [12]:
model.fit(X_train,y_train, batch_size=32, epochs=5, validation_split=0.1, verbose=1)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 1501 samples, validate on 167 samples
Epoch 1/5
1501/1501 [==============================] - 805s 536ms/step - loss: 1.0317 - accuracy: 0.6389 - val_loss: 0.4526 - val_accuracy: 0.8683
Epoch 2/5
1501/1501 [==============================] - 855s 570ms/step - loss: 0.2754 - accuracy: 0.9320 - val_loss: 0.3211 - val_accuracy: 0.9162
Epoch 3/5
1501/1501 [==============================] - 836s 557ms/step - loss: 0.1262 - accuracy: 0.9720 - val_loss: 0.1314 - val_accuracy: 0.9581
Epoch 4/5
1501/1501 [==============================] - 828s 551ms/step - loss: 0.0367 - accuracy: 0.9940 - val_loss: 0.1112 - val_accuracy: 0.9701
Epoch 5/5
1501/1501 [==============================] - 839s 559ms/step - loss: 0.0167 - accuracy: 0.9980 - val_loss: 0.0886 - val_accuracy: 0.9760


In [13]:
score = model.evaluate(X_test,y_test)
print("Accuracy = ",score[1]*100)

557/557 [==============================] - 49s 89ms/step
Accuracy =  97.12746739387512


In [1]:
import numpy as np
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from sklearn import preprocessing
import pickle

Using TensorFlow backend.
c:\users\terry\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\terry\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\terry\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16

In [2]:
maxlen=4491
from keras.models import load_model
model=load_model('textmodel.h5')
with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)
le=preprocessing.LabelEncoder()
le.classes_ = np.load('classes.npy')

In [3]:
import pickle
newtext="The PlayStation 5 is an upcoming home video game console developed by Sony Interactive Entertainment. Announced in 2019 as the successor to the PlayStation 4, it is scheduled to launch in late 2020."
p=model.predict_classes(pad_sequences(tokenizer.texts_to_sequences([newtext]),maxlen=maxlen,padding='post'))
le.inverse_transform(p)[0]

'tech'

In [22]:
model.save("textmodel.h5")